In [53]:
# 라이브러리 불러오기기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openai
import json
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from pydantic import BaseModel
from typing import List, Optional

In [54]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'data')

In [55]:
data_file_path = os.path.join(data_directory, 'filter_df.pickle')
df = pd.read_pickle(data_file_path)

In [56]:
df = df[~df['title'].str.contains('임대|야놀자|입장권|상품권|포인트|야놀|주차권|쿠폰|구매|비행기|종일권|자유이용권', na=False)]
df.reset_index(drop=True, inplace=True)

In [57]:
load_dotenv()

openai.api_key = os.getenv('OPEN_AI_KEY')

In [60]:
df['description_1'] = '제목 : ' + df['title'].astype(str) + ', 상품 게시 시간 : ' + df['post_time'].astype(str) + ', 상세 설명 : ' + df['description'].astype(str)

In [75]:
# Pydantic을 사용해 응답 데이터를 구조화할 모델을 정의합니다.
class Fields(BaseModel):
    expiration_stdate: Optional[str]
    expiration_endate: Optional[str]
    room_type: Optional[str]
    head_count: Optional[int]
    shipping_fee: Optional[int]
    transaction_method: Optional[str]
    market_price: Optional[str]
    options: Optional[str]
    parking: Optional[bool]
    check_in_time: Optional[str]
    check_out_time: Optional[str]
    stay_type: Optional[str]

# 함수를 정의하여 데이터를 요청하고 응답을 처리합니다.
def extract_fields(description):
    prompt = {
        "role": "system", 
        "content": """
        너는 데이터 필드를 추출해주는 도우미야. 데이터를 받아서 필요한 필드를 12개로 제한해 JSON 형식으로 응답할거야. 
        설명에서 필요한 12개의 필드를 정확히 추출하고 나머지 필드들은 무시해줘. 필드는 오직 12개만 있어야 하고, 추가적인 세부 필드를 포함해서는 안 돼.
        NaN이나 정보가 없으면 해당 필드를 null로 설정해줘.
        필드는 아래 12개:
        1. expiration_stdate (해당 상품의 사용기한 중 시작 날짜 YYYY-MM-DD 형식)
        2. expiration_endate (해당 상품의 사용기한 중 마감 날짜 YYYY-MM-DD 형식)
        3. room_type (객실 유형)
        4. head_count (사용 인원수, int로만 알려줘)
        5. shipping_fee (배송비, 없으면 0)
        6. transaction_method (직거래/택배거래 여부)
        7. market_price (시중 가격, int로만 알려줘)
        8. options (호텔 옵션들 항목들을 콤마로 구분)
        9. parking (주차 가능 여부, True/False)
        10. check_in_time (체크인 시간,형식 : HH:MM)
        11. check_out_time (체크아웃 시간, 형식 : HH:MM)
        12. stay_type (숙소 형태, 예시:호텔, 모텔, 리조트, 풀빌라 등)
        """
    }

    query = {
        "role": "user",
        "content": f"""
        설명에서 필요한 12개의 필드를 추출하고 JSON 형식으로 반환해줘. 단, 필드는 12개 고정이야.
        
        설명: {description}
        """
    }

    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # 또는 "gpt-4" 사용 가능
            messages=[prompt, query],
            temperature=0,
            max_tokens=1000
        )

        # 응답을 받아 JSON으로 변환
        result = response['choices'][0]['message']['content']
        result = result.strip()  # 공백 제거

        # json으로 시작되는 쓸데없는 부분을 제거
        if result.startswith("```json"):
            result = result.replace("```json", "").strip()
        if result.endswith("```"):
            result = result.replace("```", "").strip()

        # Pydantic을 사용해 응답을 구조화된 데이터로 변환
        parsed_data = Fields.parse_raw(result)
        return parsed_data.dict()  # 딕셔너리로 반환

    except Exception as e:
        print("응답 파싱 중 오류 발생:", e)
        return {field: np.nan for field in Fields.__fields__}  # NaN 값으로 채운 딕셔너리 반환


In [76]:
# 멀티쓰레딩
def process_descriptions(descriptions):
    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(extract_fields, desc) for desc in descriptions]
        
        # tqdm으로 진행률 표시
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing descriptions"):
            results.append(future.result())
    
    return results

descriptions = df['description_1'].tolist()
processed_data = process_descriptions(descriptions)

Processing descriptions: 100%|██████████| 491/491 [04:00<00:00,  2.04it/s]


In [77]:
processed_data

[{'expiration_stdate': None,
  'expiration_endate': None,
  'room_type': '투룸 슈페리어 더블',
  'head_count': 6,
  'shipping_fee': 0,
  'transaction_method': '직거래',
  'market_price': None,
  'options': '사우나',
  'parking': False,
  'check_in_time': '15:00',
  'check_out_time': '11:00',
  'stay_type': '호텔'},
 {'expiration_stdate': None,
  'expiration_endate': None,
  'room_type': 'VIP',
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '직거래',
  'market_price': None,
  'options': '수영장,루프탑,인피니티풀,온천,사우나,이벤트탕',
  'parking': False,
  'check_in_time': None,
  'check_out_time': None,
  'stay_type': '호텔'},
 {'expiration_stdate': 'null',
  'expiration_endate': '2025-06-30',
  'room_type': '테라스풀하우스A',
  'head_count': None,
  'shipping_fee': 0,
  'transaction_method': '직거래',
  'market_price': '67',
  'options': '식사크레딧 40만원 포함',
  'parking': False,
  'check_in_time': 'null',
  'check_out_time': 'null',
  'stay_type': '호텔'},
 {'expiration_stdate': '2024-10-12',
  'expiration_endate': '2024-

In [78]:
df1 = pd.DataFrame(processed_data)

In [84]:
df.shape

(491, 15)